# FASTA Sequences

### Import libraries and load dependencies:

In [ ]:
import requests  # Library for sending HTTP requests to fetch data from URLs.
import pandas as pd  # Library for data manipulation and analysis.
from bs4 import BeautifulSoup  # Library for parsing HTML and XML documents.


### Define global variables

In [ ]:
CSV_PATH = "../../data/csv/"  # Path to the folder containing CSV files.
FASTA_PATH = "../../data/fasta/"  # Path to the folder for storing FASTA files.
WILD_FASTA_PATH = f"{FASTA_PATH}wild"  # Path for wild-type FASTA files.
COMPLETE_WILD_FASTA_PATH = f"{FASTA_PATH}complete_wild"  # Path for complete wild FASTA files.
VARIANT_FASTA_PATH = f"{FASTA_PATH}variant"  # Path for variant FASTA files.
AMINO_ACIDS = {  # Dictionary mapping single-letter amino acid codes to their full names.
    'A': 'Ala',
    'C': 'Cys',
    'D': 'Asp',
    'E': 'Glu',
    'F': 'Phe',
    'G': 'Gly',
    'H': 'His',
    'I': 'Ile',
    'K': 'Lys',
    'L': 'Leu',
    'M': 'Met',
    'N': 'Asn',
    'P': 'Pro',
    'Q': 'Gln',
    'R': 'Arg',
    'S': 'Ser',
    'T': 'Thr',
    'V': 'Val',
    'W': 'Trp',
    'Y': 'Tyr',
}
AMINO_ACIDS_REVERSE = {value: key for key, value in AMINO_ACIDS.items()}  # Reverse dictionary for mapping full names to single-letter codes.


### Load datasets:

In [ ]:
wild_df = pd.read_csv(f'{CSV_PATH}wild.csv', sep=';')  # Load wild-type dataset from CSV file.
complete_wild_df = pd.read_csv(f'{CSV_PATH}complete_wild.csv', sep=';')  # Load complete wild dataset from CSV file.
variant_df = pd.read_csv(f'{CSV_PATH}variant.csv', sep=';')  # Load variant dataset from CSV file.

### Function to fetch wild FASTA sequence from MycoBrowser and save into a file

In [ ]:
def save_fasta_sequence_wild(df, path):
    # Iterate through each row in the DataFrame.
    for index, row in df.iterrows():
        print(f"Saving FASTA sequence... ({index+1} from {df.shape[0]})")  # Log progress.
        gene = row["gene"]  # Extract gene name from the current row.
        url = row["mycobrowser_url"]  # Extract Mycobrowser URL.
        response = requests.get(url)  # Send GET request to fetch HTML content.
        if response.status_code == 200:  # Check if the request was successful.
            soup = BeautifulSoup(response.text, 'html.parser')  # Parse HTML content.
            fasta_section = soup.find_all('pre')  # Find all <pre> tags, which may contain FASTA sequences.
            if fasta_section:  # Check if any <pre> tags were found.
                for section in fasta_section:
                    if "Mycobacterium tuberculosis H37Rv|" in section.text:  # Ensure the section contains the correct FASTA sequence.
                        fasta_sequence = section.text.strip()  # Get the FASTA sequence and remove extra spaces.
                        filename = f'{path}/{gene}.fasta'  # Create the file path for saving the sequence.
                        with open(filename, 'w') as f:  # Open the file in write mode.
                            f.write(fasta_sequence)  # Write the FASTA sequence to the file.
                        fasta_sequence = fasta_sequence.split("\n")[1]  # Extract only the sequence part (ignoring the header).
                        df.loc[index, "fasta"] = fasta_sequence  # Add the sequence to the DataFrame.
            else:
                print(f"FASTA sequence not available")  # Log if no FASTA section was found.
        else:
            print(f"Gene information not found")  # Log if the request failed.


### Getting FASTA sequence from Mycobrowser url for all wild type

In [ ]:
save_fasta_sequence_wild(complete_wild_df, COMPLETE_WILD_FASTA_PATH)  # Fetch and save FASTA sequences for complete wild types.

In [ ]:
print(complete_wild_df[complete_wild_df["fasta"].isnull()]) # Log genes without FASTA sequences due to their rRNA type. 

PS: There are two genes that don't have a FASTA sequence, because they are of rRNA type: `rrs` and `rrl`. So we will remove them.

In [ ]:
# Remove rows for `rrs` and `rrl` genes that lack FASTA sequences.
complete_wild_df = complete_wild_df[complete_wild_df.gene != "rrs"]
complete_wild_df = complete_wild_df[complete_wild_df.gene != "rrl"]
complete_wild_df.head()  # Display the first few rows of the DataFrame.


In [ ]:
print(f"Dataset shape: {complete_wild_df.shape}")  # Print the dataset's shape.
print(f"Rows: {complete_wild_df.shape[0]}")  # Print the number of rows.
print(f"Columns: {complete_wild_df.shape[1]}")  # Print the number of columns.


In [ ]:
complete_wild_df.describe()  # Display summary statistics.

In [ ]:
complete_wild_df.to_csv(f"{CSV_PATH}fasta_complete_wild.csv", index=False, sep=';') # Save the updated DataFrame with FASTA sequences to a CSV file.

### Getting FASTA sequence from Mycobrowser url for filtered wild type

In [ ]:
save_fasta_sequence_wild(wild_df, WILD_FASTA_PATH)  # Fetch and save FASTA sequences for filtered wild types.
wild_df.head()  # Display the first few rows of the DataFrame.


In [ ]:
print(f"Dataset shape: {wild_df.shape}")  # Print the dataset's shape.
print(f"Rows: {wild_df.shape[0]}")  # Print the number of rows.
print(f"Columns: {wild_df.shape[1]}")  # Print the number of columns.


In [ ]:
wild_df.describe()  # Display summary statistics.

In [ ]:
wild_df.to_csv(f"{CSV_PATH}fasta_wild.csv", index=False, sep=';') # Save the updated DataFrame with FASTA sequences to a CSV file.


### Getting FASTA sequence for mutations

In [ ]:
variant_df.head()  # Display the first few rows of the DataFrame.


In [ ]:
print(f"Dataset shape: {variant_df.shape}")  # Print the dataset's shape.
print(f"Rows: {variant_df.shape[0]}")  # Print the number of rows.
print(f"Columns: {variant_df.shape[1]}")  # Print the number of columns.


In [ ]:
variant_df.describe()  # Display summary statistics.


In [ ]:
# Function to mutate a FASTA sequence based on a given mutation.
def get_fasta_mutation(fasta_selvagem, mutation):
    position = int(mutation[3:-3])  # Extract the position of the mutation.
    amino_acid = fasta_selvagem[position-1]  # Get the original amino acid at the position.
    if mutation[:3] != AMINO_ACIDS[amino_acid]:  # Check if the amino acid matches the expected value.
        print(f"Error converting fasta sequence")  # Log an error if the amino acid does not match.
        return "ERROR: amino-acid is not the same as expected"  # Return an error message.
    fasta_mutated = list(fasta_selvagem)  # Convert the FASTA sequence to a list for mutation.
    fasta_mutated[position-1] = AMINO_ACIDS_REVERSE[mutation[-3:]]  # Replace the amino acid at the position.
    fasta_mutated = ''.join(fasta_mutated)  # Convert the mutated list back to a string.
    return fasta_mutated  # Return the mutated FASTA sequence.


In [ ]:
# Function to fetch and save FASTA sequences for variants.
def save_fasta_sequence_variant(df, path):
    for index, row in df.iterrows():
        print(f"Saving FASTA sequence... ({index+1} from {df.shape[0]})")  # Log progress.
        wild_row = wild_df[wild_df['gene'] == row['gene']]  # Find the corresponding wild type row.
        if wild_row.empty:  # Check if the wild type gene is not found.
            print(f"Gene {row['gene']} not found in wild_df")  # Log a warning.
            continue  # Skip to the next row.
        wild_fasta = wild_row['fasta'].values[0]  # Get the FASTA sequence for the wild type.
        variant = row['variant'].split('.')[1]  # Extract the mutation from the variant column.
        mutated_fasta = get_fasta_mutation(wild_fasta, variant)  # Generate the mutated FASTA sequence.
        df.at[index, 'fasta'] = mutated_fasta  # Add the mutated sequence to the DataFrame.
        identifier = row['identifier']  # Get the variant identifier.
        fasta_content = f">Mycobacterium tuberculosis H37Rv|{identifier}|{variant}\n{mutated_fasta}"  # Create the FASTA header and sequence.
        with open(f"{path}/{row['variant']}.fasta", "w") as f:  # Open the file in write mode.
            f.write(fasta_content)  # Write the FASTA content to the file.

        

In [ ]:
save_fasta_sequence_variant(variant_df, VARIANT_FASTA_PATH)# Fetch and save FASTA sequences for variants.

In [ ]:
variant_df.head()  # Display the first few rows of the DataFrame.

In [ ]:
print(f"Dataset shape: {variant_df.shape}")  # Print the dataset's shape.
print(f"Rows: {variant_df.shape[0]}")  # Print the number of rows.
print(f"Columns: {variant_df.shape[1]}")  # Print the number of columns.

In [ ]:
variant_df.describe() # Display summary statistics.

In [ ]:
variant_df.to_csv(f"{CSV_PATH}fasta_variant.csv", index=False, sep=';') # Save the updated DataFrame with FASTA sequences to a CSV file.